## JSON Functions
from_json() – Converts JSON string into Struct type or Map type.

to_json() – Converts MapType or Struct type to JSON string.

json_tuple() – Extract the Data from JSON and create them as a new columns.

get_json_object() – Extracts JSON element from a JSON string based on json path specified.

schema_of_json() – Create schema string from JSON string

### Create DataFrame with Column contains JSON String
let’s create DataFrame with a column contains JSON string.

In [1]:
from pyspark.sql import SparkSession,Row
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

jsonString="""{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""
df=spark.createDataFrame([(1, jsonString)],["id","value"])
df.show(truncate=False)

/usr/local/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/17 21:45:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+--------------------------------------------------------------------------+
|id |value                                                                     |
+---+--------------------------------------------------------------------------+
|1  |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+--------------------------------------------------------------------------+



### from_json()
from_json() function is used to convert JSON string into Struct type or Map type. The below example converts JSON string to Map key-value pair

In [2]:
#Convert JSON string column to Map type
from pyspark.sql.types import MapType,StringType
from pyspark.sql.functions import from_json
df2=df.withColumn("value",from_json(df.value,MapType(StringType(),StringType())))
df2.printSchema()
df2.show(truncate=False)

root
 |-- id: long (nullable = true)
 |-- value: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+---+---------------------------------------------------------------------------+
|id |value                                                                      |
+---+---------------------------------------------------------------------------+
|1  |[Zipcode -> 704, ZipCodeType -> STANDARD, City -> PARC PARQUE, State -> PR]|
+---+---------------------------------------------------------------------------+



### to_json()
to_json() function is used to convert DataFrame columns MapType or Struct type to JSON string. Using df2 that created from above from_json() example

In [3]:
from pyspark.sql.functions import to_json,col
df2.withColumn("value",to_json(col("value"))).show(truncate=False)

+---+----------------------------------------------------------------------------+
|id |value                                                                       |
+---+----------------------------------------------------------------------------+
|1  |{"Zipcode":"704","ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+----------------------------------------------------------------------------+



### json_tuple()
 json_tuple() is used the query or extract the elements from JSON column and create the result as a new columns.

In [8]:
from pyspark.sql.functions import json_tuple
from termcolor import cprint
cprint("----- df", "blue")
df.show(truncate=False)
cprint('----- df.select(col("id"),json_tuple(col("value"),"Zipcode","ZipCodeType","City")).toDF("id","Zipcode","ZipCodeType","City")', "red")
df.select(col("id"),json_tuple(col("value"),"Zipcode","ZipCodeType","City")) \
    .toDF("id","Zipcode","ZipCodeType","City") \
    .show(truncate=False)

----- df
+---+--------------------------------------------------------------------------+
|id |value                                                                     |
+---+--------------------------------------------------------------------------+
|1  |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+--------------------------------------------------------------------------+

----- df.select(col("id"),json_tuple(col("value"),"Zipcode","ZipCodeType","City")).toDF("id","Zipcode","ZipCodeType","City")
+---+-------+-----------+-----------+
|id |Zipcode|ZipCodeType|City       |
+---+-------+-----------+-----------+
|1  |704    |STANDARD   |PARC PARQUE|
+---+-------+-----------+-----------+



### get_json_object()
get_json_object() is used to extract the JSON string based on path from the JSON column.

In [7]:
from pyspark.sql.functions import get_json_object
from termcolor import cprint
cprint("----- df", "blue")
df.show(truncate=False)
cprint('----- df.select(col("id"),get_json_object(col("value"),"$.ZipCodeType").alias("ZipCodeType"))', "red")
df.select(col("id"),get_json_object(col("value"),"$.ZipCodeType").alias("ZipCodeType")) \
    .show(truncate=False)

----- df
+---+--------------------------------------------------------------------------+
|id |value                                                                     |
+---+--------------------------------------------------------------------------+
|1  |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+--------------------------------------------------------------------------+

----- df.select(col("id"),get_json_object(col("value"),"$.ZipCodeType").alias("ZipCodeType"))
+---+-----------+
|id |ZipCodeType|
+---+-----------+
|1  |STANDARD   |
+---+-----------+



### schema_of_json()
Use schema_of_json() to create schema string from JSON string column.

In [9]:
from pyspark.sql.functions import schema_of_json,lit
schemaStr=spark.range(1) \
    .select(schema_of_json(lit("""{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""))) \
    .collect()[0][0]
print(schemaStr)

struct<City:string,State:string,ZipCodeType:string,Zipcode:bigint>


### Read and flatten JSON DATA

In [3]:
data_df = spark.read.option("multiline","true").json("./resources/json_files/json4flatten.json")
data_df.show()

+--------------------+------------+--------------------+
|           addresses|contact_type|                  id|
+--------------------+------------+--------------------+
|[{HILLSBORO, null...|     Company|da38f109-8b8d-d4a...|
|[{TAMPA, US, 1284...|  individual|f109da38-d4af-576...|
+--------------------+------------+--------------------+



#### use explode to get addresses' sub-columns

In [4]:
from pyspark.sql.functions import explode

data2_df = data_df.select(
    "contact_type",
    "id",
    explode("addresses").alias("addressesExplode")
).select("contact_type","id", "addressesExplode.*")

data2_df.show()

+------------+--------------------+---------+------------+--------------------+-----------+----------+
|contact_type|                  id|     city|country_code|               line1|postal_code|state_code|
+------------+--------------------+---------+------------+--------------------+-----------+----------+
|     Company|da38f109-8b8d-d4a...|HILLSBORO|        null|     5210 16TH AVE S|      33619|        FL|
|     Company|da38f109-8b8d-d4a...|PLACENTIA|        null|3417 SANDY PORTER RD|      28273|        NC|
|     Company|da38f109-8b8d-d4a...|FAIRBANKS|         USA| 455 3RD AVE STE 120|      99701|        AK|
|  individual|f109da38-d4af-576...|    TAMPA|          US|     12840 SW RIV RD|      97123|        OR|
|  individual|f109da38-d4af-576...|PLACENTIA|        null|1760 PEACHTREE ST...|      30309|        GA|
|  individual|f109da38-d4af-576...|  ATLANTA|         USA|         107 POLK ST|      92870|        CA|
+------------+--------------------+---------+------------+---------------

restart spark

In [1]:
from pyspark.sql import SparkSession
from delta import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

builder = SparkSession.builder.appName("delta-table") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

/usr/local/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9ec065be-c57b-45c7-a1f0-2a518fa1ee2a;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.1 in central
	found io.delta#delta-storage;2.1.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 285ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.1 from central in [default]
	io.delta#delta-storage;2.1.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number

22/12/17 22:20:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
jsonString="""[{"contact_type":'Company',"id":"da38f109-8b8d-d4af-576b-64a0fd87d247","addresses":[{"city":"HILLSBORO","country_code":"US","line1":"12840 SW RIV RD","postal_code":"97123","state_code":"OR"},{"city":"ATLANTA","country_code":"USA","line1":"1760 PEACHTREE ST NW 100","postal_code":"30309","state_code":"GA"}]}]"""
# jsonString="""[{"contact_type":"Company","id":"da38f109-8b8d-d4af-576b-64a0fd87d247","addresses":[{"city":"HILLSBORO","country_code":"US","line1":"12840 SW RIV RD","postal_code":"97123","state_code":"OR"}]}]"""

df_x=spark.createDataFrame([(1, jsonString)],["index","contacts"])
df_x.show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|index|contacts                                                                                                                                                                                                                                                                                                            |
+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1    |[{"contact_type":'Company',"id":"da38f109-

In [4]:
schema_from_df = spark.read.json(df_x.rdd.map(lambda row: row["contacts"]))
schema_from_df.printSchema()

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- country_code: string (nullable = true)
 |    |    |-- line1: string (nullable = true)
 |    |    |-- postal_code: string (nullable = true)
 |    |    |-- state_code: string (nullable = true)
 |-- contact_type: string (nullable = true)
 |-- id: string (nullable = true)



In [5]:
schema_relation = schema_from_df.schema
df_y = df_x.withColumn('contacts', from_json(col('contacts'), ArrayType(schema_relation)))
df_y.show(truncate=False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|index|contacts                                                                                                                                             |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|1    |[{[{HILLSBORO, US, 12840 SW RIV RD, 97123, OR}, {ATLANTA, USA, 1760 PEACHTREE ST NW 100, 30309, GA}], Company, da38f109-8b8d-d4af-576b-64a0fd87d247}]|
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------+



In [6]:
data2_df = df_y.select(
    "index",
    explode("contacts").alias("contactsExplode")
).select("index", "contactsExplode.*")

data2_df.show()

+-----+--------------------+------------+--------------------+
|index|           addresses|contact_type|                  id|
+-----+--------------------+------------+--------------------+
|    1|[{HILLSBORO, US, ...|     Company|da38f109-8b8d-d4a...|
+-----+--------------------+------------+--------------------+



In [7]:
data3_df = data2_df.select(
    "index",
    "contact_type",
    "id",
    explode("addresses").alias("addressesExplode")
).select("index","contact_type","id", "addressesExplode.*")

data3_df.show()

+-----+------------+--------------------+---------+------------+--------------------+-----------+----------+
|index|contact_type|                  id|     city|country_code|               line1|postal_code|state_code|
+-----+------------+--------------------+---------+------------+--------------------+-----------+----------+
|    1|     Company|da38f109-8b8d-d4a...|HILLSBORO|          US|     12840 SW RIV RD|      97123|        OR|
|    1|     Company|da38f109-8b8d-d4a...|  ATLANTA|         USA|1760 PEACHTREE ST...|      30309|        GA|
+-----+------------+--------------------+---------+------------+--------------------+-----------+----------+



### Flatten function
check the schema

In [8]:
df = data2_df
df.printSchema()
df.schema.fields

root
 |-- index: long (nullable = true)
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- country_code: string (nullable = true)
 |    |    |-- line1: string (nullable = true)
 |    |    |-- postal_code: string (nullable = true)
 |    |    |-- state_code: string (nullable = true)
 |-- contact_type: string (nullable = true)
 |-- id: string (nullable = true)



[StructField('index', LongType(), True),
 StructField('addresses', ArrayType(StructType([StructField('city', StringType(), True), StructField('country_code', StringType(), True), StructField('line1', StringType(), True), StructField('postal_code', StringType(), True), StructField('state_code', StringType(), True)]), True), True),
 StructField('contact_type', StringType(), True),
 StructField('id', StringType(), True)]

In [9]:
df.show()

+-----+--------------------+------------+--------------------+
|index|           addresses|contact_type|                  id|
+-----+--------------------+------------+--------------------+
|    1|[{HILLSBORO, US, ...|     Company|da38f109-8b8d-d4a...|
+-----+--------------------+------------+--------------------+



1- get in a dictionary all elements in the schema that are ArrayType or Structype (others won't need to be flatten)

In [10]:
complex_fields = dict([(field.name, field.dataType) for field in df.schema.fields if isinstance(field.dataType, ArrayType) or isinstance(field.dataType, StructType)])
print(complex_fields) 

{'addresses': ArrayType(StructType([StructField('city', StringType(), True), StructField('country_code', StringType(), True), StructField('line1', StringType(), True), StructField('postal_code', StringType(), True), StructField('state_code', StringType(), True)]), True)}


2- get all to be flatten col names

In [11]:
qualify = list(complex_fields.keys())[0] + "_"
print(qualify)

addresses_


`explode` creates a row for each element in the array or map column by ignoring null or empty values in array whereas `explode_outer` returns all values in array or map including null or empty.

In [13]:
from termcolor import cprint

while len(complex_fields) != 0:
    col_name = list(complex_fields.keys())[0]
    cprint(f"---- {col_name} ---", 'green')

    if isinstance(complex_fields[col_name], StructType):
        cprint("instance StructType", 'cyan')
        if (type =='standard'):
            expanded = [col(col_name + '.' + k).alias(k) 
                    for k in [n.name for n in complex_fields[col_name]]
                    ]
            cprint(f"expanded {expanded}", 'blue')
        else:
            expanded = [col(col_name + '.' + k).alias(col_name + '_' + k) 
                    for k in [n.name for n in complex_fields[col_name]]
                    ]
            cprint(f"expanded {expanded}", 'red')

        df = df.select("*", *expanded).drop(col_name)
        df.show()

    elif isinstance(complex_fields[col_name], ArrayType):
        cprint("instance ArrayType", 'magenta')
        df = df.withColumn(col_name, explode_outer(col_name))
        df.show()
    
    elif (type(complex_fields[col_name]) == ArrayType):
        cprint("type ArrayType", 'yellow')
        df = df.withColumn(col_name, explode_outer(col_name))
        df.show()

    complex_fields = dict([
        (field.name, field.dataType)
        for field in df.schema.fields
        if isinstance(field.dataType, ArrayType) or isinstance(field.dataType, StructType)
    ])

    cprint(f"======== {complex_fields.get(col_name)} ==", "yellow")    
    cprint(f"=========", "yellow")

---- addresses ---
instance ArrayType
+-----+--------------------+------------+--------------------+
|index|           addresses|contact_type|                  id|
+-----+--------------------+------------+--------------------+
|    1|{HILLSBORO, US, 1...|     Company|da38f109-8b8d-d4a...|
|    1|{ATLANTA, USA, 17...|     Company|da38f109-8b8d-d4a...|
+-----+--------------------+------------+--------------------+

======== StructType([StructField('city', StringType(), True), StructField('country_code', StringType(), True), StructField('line1', StringType(), True), StructField('postal_code', StringType(), True), StructField('state_code', StringType(), True)]) ==
---- addresses ---
instance StructType
expanded [Column<'addresses.city AS addresses_city'>, Column<'addresses.country_code AS addresses_country_code'>, Column<'addresses.line1 AS addresses_line1'>, Column<'addresses.postal_code AS addresses_postal_code'>, Column<'addresses.state_code AS addresses_state_code'>]
+-----+-----------

In [14]:
for df_col_name in df.columns:
    df = df.withColumnRenamed(df_col_name, df_col_name.replace(qualify, ""))

df.show()

+-----+------------+--------------------+---------+------------+--------------------+-----------+----------+
|index|contact_type|                  id|     city|country_code|               line1|postal_code|state_code|
+-----+------------+--------------------+---------+------------+--------------------+-----------+----------+
|    1|     Company|da38f109-8b8d-d4a...|HILLSBORO|          US|     12840 SW RIV RD|      97123|        OR|
|    1|     Company|da38f109-8b8d-d4a...|  ATLANTA|         USA|1760 PEACHTREE ST...|      30309|        GA|
+-----+------------+--------------------+---------+------------+--------------------+-----------+----------+



#### Check how the loop creates and alias the col names

In [15]:
test_dict = StructType([StructField('city',StringType(),True),
                        StructField('country_code',StringType(),True),
                        StructField('line1',StringType(),True),
                        StructField('postal_code',StringType(),True),
                        StructField('state_code',StringType(),True)])            
expanded = [col('addresses'  + '.' + k).alias(k) 
                    for k in [n.name for n in test_dict]]

print(expanded)

[Column<'addresses.city AS city'>, Column<'addresses.country_code AS country_code'>, Column<'addresses.line1 AS line1'>, Column<'addresses.postal_code AS postal_code'>, Column<'addresses.state_code AS state_code'>]


In [16]:
from pyspark.sql import Row
df = spark.createDataFrame([Row(id=1, data=10), Row(id=2, data=20),Row(id=3, data=None),Row(id=4, data=40)])
df.show()

+---+----+
| id|data|
+---+----+
|  1|  10|
|  2|  20|
|  3|null|
|  4|  40|
+---+----+



In [17]:
dd = df.toJSON()

df_x = spark.read.option('multiline', 'true').json(dd)
df_x.show()

+----+---+
|data| id|
+----+---+
|  10|  1|
|  20|  2|
|null|  3|
|  40|  4|
+----+---+



In [18]:
df_sub = df.subtract(df_x)
df_sub.show()

+---+----+
| id|data|
+---+----+
|  2|  20|
|  3|null|
|  1|  10|
|  4|  40|
+---+----+



In [19]:
df_sub = df_x.subtract(df)
df_sub.show()

+----+---+
|data| id|
+----+---+
|null|  3|
|  20|  2|
|  10|  1|
|  40|  4|
+----+---+



In [20]:
df_2 = spark.createDataFrame([Row(a=[Row(b=1, c=2), Row(b=10, c=20)]), Row(a=[Row(b=5, c=None), Row(b=50, c=60)]), Row(a=[Row(b=7, c=8), Row(b=0, c=None)])])
df_2.show(truncate=False)

+---------------------+
|a                    |
+---------------------+
|[{1, 2}, {10, 20}]   |
|[{5, null}, {50, 60}]|
|[{7, 8}, {0, null}]  |
+---------------------+



In [21]:
dd_2 = df_2.toJSON()

df_x2 = spark.read.option('multiline', 'true').json(dd_2)
df_x2.show(truncate=False)

+---------------------+
|a                    |
+---------------------+
|[{1, 2}, {10, 20}]   |
|[{5, null}, {50, 60}]|
|[{7, 8}, {0, null}]  |
+---------------------+



In [22]:
df_sub2 = df_2.subtract(df_x2)
df_sub2.show()

+---+
|  a|
+---+
+---+



In [23]:
df_3 = spark.createDataFrame([Row(a=[Row(b=1, c=2), Row(b=10, c=20)]), Row(a=[Row(b=5, c=9), Row(b=50, c=60)]), Row(a=[Row(b=7, c=8), Row(b=0, c=None)])])
df_3.show(truncate=False)

+-------------------+
|a                  |
+-------------------+
|[{1, 2}, {10, 20}] |
|[{5, 9}, {50, 60}] |
|[{7, 8}, {0, null}]|
+-------------------+



In [24]:
df_sub_x = df_3.subtract(df_2)
df_sub_x.show()

+------------------+
|                 a|
+------------------+
|[{5, 9}, {50, 60}]|
+------------------+

